In [1]:
import torch

In [2]:
X = torch.rand(4,3)
Y = torch.rand(4,3)
###행으로 변합 코드
raw = torch.cat([X,Y], dim=0) 
###열로 병합 코드
col = torch.cat([X,Y], dim=1)

In [3]:
!pip install d2l

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 82 kB 591 kB/s 
     |████████████████████████████████| 11.2 MB 11.4 MB/s 
     |████████████████████████████████| 61 kB 6.9 MB/s 
     |████████████████████████████████| 9.9 MB 40.5 MB/s 
     |████████████████████████████████| 15.7 MB 43.7 MB/s 
     |████████████████████████████████| 944 kB 59.6 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    

In [5]:
%matplotlib inline
import random
import torch
from d2l import torch as d2l

In [6]:
def synthetic_data(w, b, num_examples):
    """Generate y = Xw + b + noise."""
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([3, -1.5])
true_b = 5.5
features, labels = synthetic_data(true_w, true_b, 2000)

In [7]:
print('features:', features[0],'\nlabel:', labels[0])

features: tensor([-0.3920, -0.7988]) 
label: tensor([5.5329])


In [11]:
def data_iter(batch_size, features, labels): #최적화를 효율적으로 하기 위해 배치 사이즈만큼 여러 샘플들을 iteration
  num_examples = len(features)
  indices = list(range(num_examples))
  random.shuffle(indices)
  for i in range(0, num_examples, batch_size):
    batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
    yield features[batch_indices], labels[batch_indices] #yield는 return과 비슷함

In [12]:
batch_size = 20

for X, y in data_iter(batch_size, features, labels):
    print(X, '\n', y)
    break

tensor([[ 0.9826, -0.5920],
        [-1.9362,  0.7750],
        [ 0.4069,  0.8297],
        [ 0.6504,  1.4808],
        [ 0.5917, -0.1267],
        [-1.8422,  0.0506],
        [ 0.1861, -1.0879],
        [-0.7913, -0.3509],
        [-0.9954,  0.3850],
        [-0.0660,  1.0588],
        [ 1.3679,  0.5134],
        [ 1.1917,  0.5954],
        [-0.6672,  0.0919],
        [-1.6599, -1.2882],
        [ 0.7201, -0.3072],
        [-0.4494, -1.2879],
        [-0.1875, -1.1513],
        [-0.0531, -2.1420],
        [ 1.5295,  0.4074],
        [-0.6008, -0.6560]]) 
 tensor([[ 9.3289],
        [-1.4743],
        [ 5.4677],
        [ 5.2121],
        [ 7.4382],
        [-0.1108],
        [ 7.6866],
        [ 3.6766],
        [ 1.9351],
        [ 3.7075],
        [ 8.8258],
        [ 8.1989],
        [ 3.3801],
        [ 2.4621],
        [ 8.1300],
        [ 6.0782],
        [ 6.6699],
        [ 8.5662],
        [ 9.4719],
        [ 4.6857]])


In [13]:
#평균값이 0이고 표준편차가 0.01인 정규분포를 따르는 랜덤한 수로 가중치 초기화
#bias = 0으로 설정
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [22]:
#모델 정의
def linreg(X, w, b):
  return torch.matmul(X,w) + b #w를 곱하고 offset(b)를 더한 선형 모델 정의 #차원 에러 때문에 dot -> matmul

In [16]:
#loss function 정의
def squared_loss(y_hat, y):  #실제값 y와 예측값 y_hat의 모양을 동일하게 변형
    """Squared loss."""
    return (y_hat-y.reshape(y_hat.shape))**2 / 2  #return 값은 y_hat의 shape과 동일하게 바꿈

In [20]:
def sgd(params, lr, batch_size): #경사하강법 #손실을 줄이는 방향으로 파라미터를 업데이트
  with torch.no_grad():
    for param in params:
      param -= lr * param.grad / batch_size #평균을 구하기 위해 gradient의 합을 배치 사이즈로 나눔, 학습 속도에 비례해 업데이트
      param.grad.zero_()

In [23]:
lr = 0.03 # learning rate
num_epochs = 3 #iteration 수
net = linreg #앞서 정의한 모델
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y)
        # Compute gradient on `l` with respect to [`w`, `b`]
        l.sum().backward()
        sgd([w,b], lr, batch_size) #gradient 이용해 파라미터 업데이트
    with torch.no_grad():
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.048913
epoch 2, loss 0.000167
epoch 3, loss 0.000050


In [24]:
print(f'error in estimating w: {true_w - w.reshape(true_w.shape)}')
print(f'error in estimating b: {true_b - b}')

error in estimating w: tensor([-9.7036e-05,  4.4227e-04], grad_fn=<SubBackward0>)
error in estimating b: tensor([0.0002], grad_fn=<RsubBackward1>)


#cost graph 그리기 (x축 w, y축 loss)

In [43]:
import numpy as np
import matplotlib.pyplot as plt

mse_list = []
w_list = []

for w in np.arange(0.0, 4.1, 0.1): #이 범위는 뭐람
  print("weight =", w)
  loss_sum = 0
  for x_val, y_val in zip(X, y):
    y_pred_val = forward(X)
    l = squard_loss(X, y)
    l_sum += l
    print("\t", X, y, y_pred_val, l)
    w_list.append(w)
    mse_list.append(l_sum)

plt.plot(w_list, mse_list)
plt.ylabel('Loss')
plt.xlabel('weight')
plt.show

weight = 0.0


NameError: ignored

#concise implementation of linear regression 
- api 사용해보기

In [25]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l


true_w = torch.tensor([3, -1.5])
true_b = 5.5
features, labels = d2l.synthetic_data(true_w, true_b, 2000)

In [26]:
def load_array(data_arrays, batch_size, is_train=True): 
    """Construct a PyTorch data iterator."""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 20
data_iter = load_array((features, labels), batch_size)

In [27]:
next(iter(data_iter))

[tensor([[ 0.4634,  1.5559],
         [ 1.9405, -0.8560],
         [-0.3897,  0.6014],
         [ 0.8250,  0.0148],
         [ 1.4220,  0.7943],
         [-0.0747, -0.9525],
         [-1.2512, -1.1057],
         [-0.9378,  0.8586],
         [-0.2902, -1.8832],
         [ 0.4109, -0.6452],
         [ 1.4583,  0.7663],
         [-0.8754, -1.1946],
         [ 1.0441,  0.2145],
         [-0.0714, -0.4519],
         [-1.2354, -0.3715],
         [ 0.7636, -1.0518],
         [ 0.8319, -0.3404],
         [ 2.1367, -1.3192],
         [ 1.4655, -2.2171],
         [ 0.5746, -0.7088]]), tensor([[ 4.5387],
         [12.5973],
         [ 3.4296],
         [ 7.9424],
         [ 8.5657],
         [ 6.6991],
         [ 3.3974],
         [ 1.3893],
         [ 7.4485],
         [ 7.7033],
         [ 8.7261],
         [ 4.6609],
         [ 8.3327],
         [ 5.9647],
         [ 2.3487],
         [ 9.3529],
         [ 8.5096],
         [13.8970],
         [13.2140],
         [ 8.2729]])]

In [29]:
from torch import nn #nn(neural network)

net = nn.Sequential(nn.Linear(2,1)) #nn.Linear 선형모델 구현

In [30]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [31]:
loss = nn.MSELoss()

In [34]:
import torch.optim as optim
trainer = torch.optim.SGD(net.parameters(), lr = 0.03) 

In [35]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y) #forward pass를 수행해서 예측값 net(X)를 생성하고, 손실 l을 계산
        trainer.zero_grad()
        l.backward() #backward pass를 실행해 gradient 계산
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000232
epoch 2, loss 0.000096
epoch 3, loss 0.000096


In [37]:
w = net[0].weight.data #net 인스턴스로 층을 얻어냄 -> 가중치와 편향 변수에 접근
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([ 0.0002, -0.0001])
error in estimating b: tensor([-0.0003])
